In [1]:
# Import Packages
import pandas as pd
import timeit
import pmdarima as pm

ImportError: No module named pmdarima

In [2]:
file="C:\\Users\\kasthurirangan.sampa\\Downloads\\dashboard refresh\\applemobilitytrends-2020-07-18.csv"
data=pd.read_csv(file)
temp1 = pd.melt(data, id_vars=['geo_type', 'region', 'transportation_type', 'alternative_name',
       'sub-region', 'country'], var_name='Date', value_name='Value')
temp1=temp1[(temp1["geo_type"]=="sub-region")&(temp1["country"]=="United States")]
temp1["Date"]=pd.to_datetime(temp1['Date'])
temp2=temp1.pivot(index='Date',columns='region', values='Value')

In [7]:
max_date=max(temp1['Date']) - pd.DateOffset(4)
max1_Date=max(temp1['Date'])+ pd.DateOffset(6)
temp1["Date"]=pd.to_datetime(temp1['Date'])

In [ ]:
output=pd.DataFrame()
start = timeit.default_timer()
for region in temp1.region.unique():
#for region in region_list:
    print("\n Region:",region)
    temp=temp1[temp1["region"]==region]
    temp2=temp[temp['Date']<=max_date]
    temp2=temp2[['Date', 'Value']]
    temp2 = temp2.fillna(method = 'ffill')
    temp2.set_index("Date", inplace = True) 
    model = pm.auto_arima(temp2["Value"],
                      m = 10, seasonal = True,
                      start_p = 0,start_q=0,max_order=4,test='adf',error_action = 'ignore',
                      suppress_warnings =True,
                      stepwise =True,trace = True)
    forecast = model.predict(n_periods=10,return_conf_int=True)
    forecast_range = pd.date_range(start=max1_Date,periods=10,freq='D')
    lower = pd.Series(forecast[1][:,0],index=forecast_range)
    upper = pd.Series(forecast[1][:,1],index=forecast_range)
    forecast_df = pd.DataFrame(forecast[0],index=forecast_range,columns=['Apple_mobility_Prediction'])
    forecast_df['Apple_mobility_lower']=lower
    forecast_df['Apple_mobility_upper']=upper
    forecast_df['Apple_mobility_region']=region
    output=output.append(forecast_df)
stop = timeit.default_timer()
Total_Time=(stop - start)/60 
print('Time: '+str(Total_Time)+' Minutes')

In [ ]:
output.to_csv("C:\\Users\\kasthurirangan.sampa\\Downloads\\applemobilitytrends_predicted-2020-07-12.csv")

In [ ]:
file="C:\\Users\\kasthurirangan.sampa\\Downloads\\Global_Mobility_Report.csv"
data=pd.read_csv(file)
data=data[(data["country_region_code"]=='US')&(~data["sub_region_1"].isnull())&(data["sub_region_2"].isnull())]
metric_columns=['retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'residential_percent_change_from_baseline']
data['date']=pd.to_datetime(data['date'])

In [ ]:
output=pd.DataFrame()
start = timeit.default_timer()
for region in data.sub_region_1.unique():
#for region in region_list:
    for metric in metric_columns:
        print("\n Region: "+region+"Metric: "+metric)
        temp=data[data["sub_region_1"]==region]
        temp2=temp[temp['date']<=max_date]
        temp2=temp2[['date', metric]]
        temp2 = temp2.fillna(method = 'ffill')
        temp2.set_index("date", inplace = True) 
        model = pm.auto_arima(temp2[metric],
                          m = 10, seasonal = True,
                          start_p = 0,start_q=0,max_order=4,test='adf',error_action = 'ignore',
                          suppress_warnings =True,
                          stepwise =True,trace = True)
        forecast = model.predict(n_periods=10,return_conf_int=True)
        forecast_range = pd.date_range(start=max1_Date,periods=10,freq='D')
        lower = pd.Series(forecast[1][:,0],index=forecast_range)
        upper = pd.Series(forecast[1][:,1],index=forecast_range)
        if metric=="retail_and_recreation_percent_change_from_baseline":
            forecast_df = pd.DataFrame(forecast[0],index=forecast_range,columns=[metric+'_Prediction'])
        else:
            forecast_df[metric+'_Prediction']=pd.Series(forecast[0],index=forecast_range)
        forecast_df[metric+'_lower']=lower
        forecast_df[metric+'_upper']=upper
    forecast_df['sub_region_1']=region
        #forecast_df['metric']=metric
    output=output.append(forecast_df)
stop = timeit.default_timer()
Total_Time=(stop - start)/60 
print('Time: '+str(Total_Time)+' Minutes')
output.to_csv("C:\\Users\\kasthurirangan.sampa\\Downloads\\Googlemobilitytrends_predicted-2020-07-14.csv")